# Скачивание библиотек

In [1]:
!pip install catboost

  Using cached catboost-1.2-cp39-cp39-win_amd64.whl (101.0 MB)
  Using cached graphviz-0.20.1-py3-none-any.whl (47 kB)


In [1]:
import zipfile
import os
import shutil
import pandas as pd
import numpy as np

from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score

# Чтение и обработка данных

In [2]:
DIRECTORY = 'C:/Users/pasin/DS/Лаба/РискПрофиль/'

In [3]:
# #Для Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
#Разархивирование, не запускать если папка data уже существует
with zipfile.ZipFile(f'{DIRECTORY}data.zip', 'r') as zipFile:
    zipFile.extractall(DIRECTORY)
shutil.rmtree('__MACOSX')
for mode in ['train', 'test']:
    dealsList = os.listdir(f'{DIRECTORY}data/{mode}/{mode}_deals')
    for deals in dealsList:
        with zipfile.ZipFile(f'{DIRECTORY}data/{mode}/{mode}_deals/{deals}', 'r') as zipFile:
            zipFile.extractall(f'{DIRECTORY}data/{mode}/{mode}_deals')
        os.remove(f'{DIRECTORY}data/{mode}/{mode}_deals/{deals}')

In [ ]:
for mode in ['train', 'test']:
    main = pd.read_csv(f'{DIRECTORY}data/{mode}/{mode}.csv', sep = ',')
    for clmn in ['start_sum', 'income_rub', 'income_percent']:
        main[clmn] = str(main[clmn]).replace(',', '.')
        main[clmn] = str(main[clmn]).replace(' ', '')
    main['income_percent'].loc[main['income_percent'] == '-'] = np.nan
    main = main.astype({'start_sum': 'float', 'income_rub': 'float', 'income_percent': 'float'})
    os.remove(f'{DIRECTORY}data/{mode}/{mode}.csv')
    main.to_csv(f'{DIRECTORY}data/{mode}/{mode}.csv', index = False)

In [11]:
def formatAccountCondition (id: int, mode: str):
  try:
    account_condition = pd.read_csv(f'{DIRECTORY}data/{mode}/{mode}_additional_info/id_{id}/account_condition_{id}.csv', sep = ';', index_col = 0)
  except FileNotFoundError:
    print('No such file for id', id)
    return None
  account_condition['date'] = pd.to_datetime(account_condition['date'], format='%Y-%m-%d')
  for clmn in ['start_sum', 'warranty_provision', 'free_funds']:
    account_condition[clmn].loc[account_condition[clmn] == '-'] = np.nan
    if account_condition[clmn].isna().sum() != account_condition.shape[0]:
      account_condition[[clmn, f'{clmn}_delt']] = account_condition[clmn].str.split('(', expand = True)
      account_condition[f'{clmn}_delt'] = account_condition[f'{clmn}_delt'].str.replace(')', '')
      account_condition[f'{clmn}_delt'] = account_condition[f'{clmn}_delt'].str.replace('+', '')
      account_condition[f'{clmn}_delt'].loc[account_condition[f'{clmn}_delt'] == '-'] = np.nan
    else:
      account_condition[f'{clmn}_delt'] = np.nan
  account_condition = account_condition.astype({'start_sum': 'float', 'warranty_provision': 'float', 'free_funds': 'float', 'start_sum_delt': 'float', 'warranty_provision_delt': 'float', 'free_funds_delt': 'float'})
  os.remove(f'{DIRECTORY}data/{mode}/{mode}_additional_info/id_{id}/account_condition_{id}.csv')
  account_condition.to_csv(f'{DIRECTORY}data/{mode}/{mode}_additional_info/id_{id}/account_condition_{id}.csv')
  return account_condition
  

In [12]:
def formatReferencePoint (id: int, mode: str):
  try:
    reference_point = pd.read_csv(f'{DIRECTORY}data/{mode}/{mode}_additional_info/id_{id}/reference_point_{id}.csv', sep = ';', index_col = 0)
  except FileNotFoundError:
    print('No such file for id', id)
    return None
  for clmn in ['market', 'ticker', 'open_positions', 'price', 'end_day_balance', 'estimated_cost']:
    reference_point[clmn].loc[reference_point[clmn] == '-'] = np.nan
  if reference_point['open_positions'].isna().sum() != reference_point.shape[0]:
    reference_point[['open_positions', 'open_positions_delt']] = reference_point['open_positions'].str.split('(', expand = True)
    reference_point['open_positions_delt'] = reference_point['open_positions_delt'].str.replace(')', '')
    reference_point['open_positions_delt'] = reference_point['open_positions_delt'].str.replace('+', '')
    reference_point['open_positions_delt'].loc[reference_point['open_positions_delt'] == '-'] = np.nan
  else:
    reference_point['open_positions_delt'] = np.nan
  reference_point = reference_point.astype({'open_positions': 'float', 'price': 'float', 'end_day_balance': 'float', 'estimated_cost': 'float'})
  os.remove(f'{DIRECTORY}data/{mode}/{mode}_additional_info/id_{id}/reference_point_{id}.csv')
  reference_point.to_csv(f'{DIRECTORY}data/{mode}/{mode}_additional_info/id_{id}/reference_point_{id}.csv')
  return reference_point

In [13]:
def addNamesToDeals (id: int, market: int, mode: str):
    try:
        deals = pd.read_csv(f'{DIRECTORY}data/{mode}/{mode}_deals/{market}_{id}.csv', names = ["datetime", "ticker", "quantity", "summ"], sep = ';')
    except FileNotFoundError:
        print('No such file for id', id)
        return None
    os.remove(f'{DIRECTORY}data/{mode}/{mode}_deals/{market}_{id}.csv')
    deals.to_csv(f'{DIRECTORY}data/{mode}/{mode}_deals/{market}_{id}.csv')
    return None

In [15]:
for mode in ['train', 'test']:
    main = pd.read_csv(f'{DIRECTORY}data/{mode}/{mode}.csv', sep = ',')
    ids = main['id']
    for id in ids:
        formatAccountCondition(id, mode)
        formatReferencePoint(id, mode)
        for i in range(1, 4):
            addNamesToDeals(id, i, mode)

KeyError: 'date'

In [ ]:
# reading data
train = pd.read_csv(f'{DIRECTORY}data/train/train.csv', sep = ',')
final_test = pd.read_csv(f'{DIRECTORY}data/test/test.csv', sep = ',')

# Обучение моделей

In [ ]:
# splitting train on train and test sets
X = train.drop(["class", "nickname", "id"], 1)
y = train["class"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size = 0.75)

СatBoostСlassifier 

In [ ]:
%%time

cat_columns = X_train.select_dtypes(include=['object']).columns
model = CatBoostClassifier(cat_features = [cat_columns[0]], verbose=0, loss_function='MultiClass')

param_grid = {'iterations': [100, 500],
              'learning_rate': [0.01, 0.05, 0.1],
              'depth': [4, 5, 7], 
              "l2_leaf_reg": [0.5, 1, 4, 8]}

grid_search = RandomizedSearchCV(estimator=model, 
                                 param_distributions=param_grid, 
                                 cv=5, 
                                 scoring='accuracy', 
                                 error_score='raise', 
                                 n_iter=15)

grid_search.fit(X_train, y_train, cat_features = [cat_columns[0]])

print("Best Parameters: ", grid_search.best_params_)
print("Best CV Score: ", grid_search.best_score_)

In [ ]:
results = grid_search.cv_results_
results['mean_test_score']

Метрики

In [ ]:
def metrics(y_true, y_pred):
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    f1 = f1_score(y_true, y_pred, average = 'micro')
    return f"\nrmse: {rmse}\nf1score: {f1} \n"

In [ ]:
cbc = grid_search.best_estimator_
y_pred_ts = cbc.predict(X_test)
y_pred_tr = cbc.predict(X_train)

print(f"TRAIN: {metrics(y_train, y_pred_tr)}\nTEST: {metrics(y_test, y_pred_ts)}")

# Мусорка

In [ ]:
test_additional_info_id_account_condition_list = []
test_additional_info_id_reference_point_list = []
test_additional_info_id_stats_table_list = []
test_deals_list = []

account_condition_list = []
reference_point_list = []
stats_table_list = []
test_deals_list = []
list_cnt_tables = []

main = pd.read_csv(f'/content/data/test/test.csv', sep = ',')
ids = main['id']

cnt_tables = 0
for id in ids:
  test_additional_info_id_account_condition_list.append(pd.read_csv(f'/content/data/test/test_additional_info/id_{id}/account_condition_{id}.csv'))
  test_additional_info_id_reference_point_list.append(pd.read_csv(f'/content/data/test/test_additional_info/id_{id}/reference_point_{id}.csv'))
  test_additional_info_id_stats_table_list.append(pd.read_csv(f'/content/data/test/test_additional_info/id_{id}/stats_table_{id}.csv', sep=';'))
  if not (pd.read_csv(f'/content/data/test/test_additional_info/id_{id}/account_condition_{id}.csv').empty):
    account_condition_list.append(1)
    cnt_tables += 1
  if not (pd.read_csv(f'/content/data/test/test_additional_info/id_{id}/reference_point_{id}.csv').empty):
    reference_point_list.append(1)
    cnt_tables += 1
  if not (pd.read_csv(f'/content/data/test/test_additional_info/id_{id}/stats_table_{id}.csv').empty):
    test_deals_list.append(1)
    cnt_tables += 1
  list_cnt_tables.append(cnt_tables)
  cnt_tables = 0

test_additional_info_id_account_condition = pd.concat(test_additional_info_id_account_condition_list, axis=0, join='outer', ignore_index=False, keys=ids)
test_additional_info_id_reference_point = pd.concat(test_additional_info_id_reference_point_list, axis=0, join='outer', ignore_index=False, keys=ids)
test_additional_info_id_stats_table = pd.concat(test_additional_info_id_stats_table_list, axis=0, join='outer', ignore_index=False, keys=ids)

# for id in ids:
#   test_deals_list.append(pd.read_csv(f'/content/data/test/test_deals/1_{id}.csv'))
# test_deals = pd.concat(test_deals_list, axis=0, join='outer', ignore_index=False, keys=ids)

distr_tables = pd.DataFrame({'id' : ids, 
                             'count_tables' : list_cnt_tables, 
                             })

distr_tables

In [ ]:
test_additional_info_id_account_condition.head()

In [ ]:
# много пропусков
test_additional_info_id_account_condition.info()

In [ ]:
# много пропусков
test_additional_info_id_reference_point.info()

In [ ]:
# много пропусков
test_additional_info_id_stats_table.info()

In [ ]:
test_additional_info_id_account_condition.sum(level='id')